In [1]:
%pip install pandas
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pymongo import MongoClient
import pandas as pd

# Crea una SparkSession
spark = SparkSession.builder \
    .appName("MongoSparkConnector") \
    .getOrCreate()

# Conecta a MongoDB usando pymongo
client = MongoClient("mongodb://172.17.0.3:27017/")
db = client['docker']
collection = db['yt-data']

# Extrae los datos desde MongoDB
mongo_data = list(collection.find())

#print(mongo_data)

# Convierte los datos a un DataFrame de pandas
pdf = pd.DataFrame(mongo_data)

# Elimina la columna '_id' si es necesario, ya que no es serializable por defecto en Spark
if '_id' in pdf.columns:
    pdf = pdf.drop(columns=['_id'])

# Convierte el DataFrame de pandas a un DataFrame de Spark
df = spark.createDataFrame(pdf)

print(pdf)
# Muestra el esquema del DataFrame
#df.printSchema()

# Muestra algunas filas del DataFrame
#df.show()

# Realiza alguna transformación en el DataFrame
#df_filtered = df.filter(df['age'] > 25)

# Muestra las filas filtradas
#df_filtered.show()

# Escribe datos de vuelta a MongoDB (opcional)
#df_filtered.write.format("mongo").mode("append").save()

# Finaliza la sesión de Spark
spark.stop()



                                                titulo  \
0                                        Amaia - Nanai   
1    Amaia - Santos Que Yo Te Pinte (Live | Vevo St...   
2    Amaia - La Vida Imposible (Live | Vevo Studio ...   
3    Amaia, Aitana - La Canción Que No Quiero Cantarte   
4                          Amaia - Bienvenidos Al Show   
..                                                 ...   
256                  Nicki Nicole, Cazzu - Cómo Dímelo   
257                     Nicki Nicole - Nos Encontramos   
258     Nicki Nicole - F**cking Diablo (Video Oficial)   
259          Nicki Nicole - Años Luz ✨ (Video Oficial)   
260      Nicki Nicole - Wapo Traketero (Video Oficial)   

                        views           fecha            user  
0       393 K visualizaciones  hace 3 semanas    @amaiaromero  
1       158 K visualizaciones     hace 2 años    @amaiaromero  
2       193 K visualizaciones     hace 2 años    @amaiaromero  
3    2,3 M de visualizaciones     hace 2 años  

In [3]:

# Reemplazar la coma por un punto en la columna 'visualizaciones'
pdf['views'] = pdf['views'].str.replace(',', '.')

# Mostrar el DataFrame resultante
print(pdf)

                                                titulo  \
0                                        Amaia - Nanai   
1    Amaia - Santos Que Yo Te Pinte (Live | Vevo St...   
2    Amaia - La Vida Imposible (Live | Vevo Studio ...   
3    Amaia, Aitana - La Canción Que No Quiero Cantarte   
4                          Amaia - Bienvenidos Al Show   
..                                                 ...   
256                  Nicki Nicole, Cazzu - Cómo Dímelo   
257                     Nicki Nicole - Nos Encontramos   
258     Nicki Nicole - F**cking Diablo (Video Oficial)   
259          Nicki Nicole - Años Luz ✨ (Video Oficial)   
260      Nicki Nicole - Wapo Traketero (Video Oficial)   

                        views           fecha            user  
0       393 K visualizaciones  hace 3 semanas    @amaiaromero  
1       158 K visualizaciones     hace 2 años    @amaiaromero  
2       193 K visualizaciones     hace 2 años    @amaiaromero  
3    2.3 M de visualizaciones     hace 2 años  

In [4]:
# Definir una función que limpie y transforme los valores de la columna
def transformar_visualizaciones(valor):
    # Obtener la parte numérica y el sufijo
    num, sufijo = valor.split(' ')[0], valor.split(' ')[1]
    
    # Transformar el valor basado en el sufijo
    if sufijo == 'M':
        return str(float(num) * 1e6).replace('.0', '')  # Multiplicar por 1 millón y convertir a string
    if sufijo == 'K':
        return str(float(num) * 1e3).replace('.0', '')  # Multiplicar por mil y convertir a string
    else:
        return num  # En caso de que haya otros sufijos en el futuro

# Aplicar la función a la columna
pdf['views'] = pdf['views'].apply(transformar_visualizaciones)

# Mostrar el DataFrame resultante
print(pdf)

                                                titulo      views  \
0                                        Amaia - Nanai     393000   
1    Amaia - Santos Que Yo Te Pinte (Live | Vevo St...     158000   
2    Amaia - La Vida Imposible (Live | Vevo Studio ...     193000   
3    Amaia, Aitana - La Canción Que No Quiero Cantarte    2300000   
4                          Amaia - Bienvenidos Al Show    1700000   
..                                                 ...        ...   
256                  Nicki Nicole, Cazzu - Cómo Dímelo   38000000   
257                     Nicki Nicole - Nos Encontramos   10000000   
258     Nicki Nicole - F**cking Diablo (Video Oficial)   15000000   
259          Nicki Nicole - Años Luz ✨ (Video Oficial)   69000000   
260      Nicki Nicole - Wapo Traketero (Video Oficial)  161000000   

              fecha            user  
0    hace 3 semanas    @amaiaromero  
1       hace 2 años    @amaiaromero  
2       hace 2 años    @amaiaromero  
3       hace 2 años

In [ ]:

# Reemplazar la coma por un punto en la columna 'visualizaciones'
pdf['views'] = pdf['views'].str.replace(',', '.')

# Mostrar el DataFrame resultante
print(pdf)